In [1]:
from __future__ import print_function
import os
from glob import glob
import numpy as np
# from pixmappy.pixmappy import DESMaps, Gnomonic, Tweak, DECamTweak
from astropy.io import fits
from astropy.time import Time
from astropy.coordinates.angles import Angle
from astropy import units as u
from astrometry.util.fits import *
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from random import randint
# from legacypipe.py.legacypipe.survey import radec_at_mjd
from scipy.optimize import leastsq, minimize, curve_fit
from scipy.interpolate import RectBivariateSpline
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('py_files'), '..')))
from py_files import create_brick_catalogue
from py_files import brick_ringmaps
from py_files import brick_DCR
from py_files import brick_lateralmaps

%load_ext autoreload
%autoreload 2

In [2]:
old_dir = '/pscratch/sd/d/dstn/forced-motions-dr10/gaia-stars/'
dcr_dir = '/pscratch/sd/n/nelfalou/dcr-gaia-stars/'

In [ ]:
brick_DCR.create_corrected_gaia(dcr_dir, old_dir)

gaia-forced-009.fits


In [6]:
fns = glob(dcr_dir + 'gaia-*.fits')
fns.sort()
print(len(fns))

FF = []
for fn in fns:
    forced = fits_table(fn)
    print(len(forced), 'Gaia from', fn)
    FF.append(forced)
data = merge_tables(FF)

45
8397107 Gaia from /pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-000.fits
8844067 Gaia from /pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-001.fits
8657094 Gaia from /pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-002.fits
8649862 Gaia from /pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-003.fits
8779493 Gaia from /pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-004.fits
13543942 Gaia from /pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-005.fits
17260643 Gaia from /pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-006.fits
12154819 Gaia from /pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-007.fits
12738131 Gaia from /pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-008.fits
13456283 Gaia from /pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-009.fits
13673946 Gaia from /pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-010.fits
11981229 Gaia from /pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-011.fits
12033642 Gaia from /pscratch/sd/n/nelfalou/dcr-gaia-st

In [16]:
len(data)

461393838

In [3]:
rm_dir = '/pscratch/sd/n/nelfalou/rm-gaia-stars/'

In [8]:
brick_ringmaps.make_ringmaps(data, rm_dir + 'ringmaps.fits')

In [11]:
rmaps = brick_ringmaps.get_ringmaps(rm_dir + 'ringmaps.fits')
tweaks = brick_ringmaps.make_splines(rmaps)
# brick_ringmaps.create_corrected_data(rm_dir, dcr_dir, tweaks, cont=True)

In [15]:
for filename in os.listdir(dcr_dir):
    f = os.path.join(dcr_dir, filename)
    print(f) 
    brick_ringmaps.create_brick_corrected_data(filename, rm_dir, dcr_dir, tweaks)

/pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-008.fits
/pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-041.fits
/pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-020.fits
/pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-033.fits
/pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-005.fits
/pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-007.fits
/pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-038.fits
/pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-013.fits
/pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-024.fits
/pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-035.fits
/pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-004.fits
/pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-014.fits
/pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-040.fits
/pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-019.fits
/pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-015.fits
/pscratch/sd/n/nelfalou/dcr-gaia-stars/gaia-forced-022.fits
/pscratch/sd/n/nelfalou/dcr-gaia-stars/g

In [4]:
fns = glob(rm_dir + 'gaia-*.fits')
fns.sort()
print(len(fns))

FF = []
TT = []
for fn in fns:
    forced = fits_table(fn)
    print(len(forced), 'Gaia from', fn)
    tractor = fits_table(old_dir + 'gaia-tractor-' + fn[-8:])

    FF.append(forced)
    TT.append(tractor)
data = merge_tables(FF)
T = merge_tables(TT)

objmap = dict([((bid,oid),i) for i,(bid,oid) in enumerate(zip(T.brickid, T.objid))])
data.t_index = np.array([objmap.get((bid,oid), -1) for bid,oid in zip(data.brickid, data.objid)])

45
8397107 Gaia from /pscratch/sd/n/nelfalou/rm-gaia-stars/gaia-forced-000.fits
8844067 Gaia from /pscratch/sd/n/nelfalou/rm-gaia-stars/gaia-forced-001.fits
8657094 Gaia from /pscratch/sd/n/nelfalou/rm-gaia-stars/gaia-forced-002.fits
8649862 Gaia from /pscratch/sd/n/nelfalou/rm-gaia-stars/gaia-forced-003.fits
8779493 Gaia from /pscratch/sd/n/nelfalou/rm-gaia-stars/gaia-forced-004.fits
13543942 Gaia from /pscratch/sd/n/nelfalou/rm-gaia-stars/gaia-forced-005.fits
17260643 Gaia from /pscratch/sd/n/nelfalou/rm-gaia-stars/gaia-forced-006.fits
12154819 Gaia from /pscratch/sd/n/nelfalou/rm-gaia-stars/gaia-forced-007.fits
12738131 Gaia from /pscratch/sd/n/nelfalou/rm-gaia-stars/gaia-forced-008.fits
13456283 Gaia from /pscratch/sd/n/nelfalou/rm-gaia-stars/gaia-forced-009.fits
13673946 Gaia from /pscratch/sd/n/nelfalou/rm-gaia-stars/gaia-forced-010.fits
11981229 Gaia from /pscratch/sd/n/nelfalou/rm-gaia-stars/gaia-forced-011.fits
12033642 Gaia from /pscratch/sd/n/nelfalou/rm-gaia-stars/gaia-forc

In [5]:
T.columns

<bound method tabledata.columns of <tabledata object with 18017918 rows and 235 columns: release, brickid, brickname, objid, brick_primary, maskbits, fitbits, type, ra, dec, ra_ivar, dec_ivar, bx, by, dchisq, ebv, mjd_min, mjd_max, ref_cat, ref_id, pmra, pmdec, parallax, pmra_ivar, pmdec_ivar, parallax_ivar, ref_epoch, gaia_phot_g_mean_mag, gaia_phot_g_mean_flux_over_error, gaia_phot_g_n_obs, gaia_phot_bp_mean_mag, gaia_phot_bp_mean_flux_over_error, gaia_phot_bp_n_obs, gaia_phot_rp_mean_mag, gaia_phot_rp_mean_flux_over_error, gaia_phot_rp_n_obs, gaia_phot_variable_flag, gaia_astrometric_excess_noise, gaia_astrometric_excess_noise_sig, gaia_astrometric_n_obs_al, gaia_astrometric_n_good_obs_al, gaia_astrometric_weight_al, gaia_duplicated_source, gaia_a_g_val, gaia_e_bp_min_rp_val, gaia_phot_bp_rp_excess_factor, gaia_astrometric_sigma5d_max, gaia_astrometric_params_solved, flux_g, flux_r, flux_i, flux_z, flux_w1, flux_w2, flux_w3, flux_w4, flux_ivar_g, flux_ivar_r, flux_ivar_i, flux_ivar_

In [6]:
dc = (T.flux_g - T.flux_i)[data.t_index]

In [ ]:
# fns = glob(old_dir + 'gaia-tractor-*.fits')
# fns.sort()
# print(len(fns))

# TT = []
# for fn in fns:
#     tractor = fits_table(fn)
#     print(len(tractor), 'Gaia from', fn)
#     TT.append(tractor)
# T = merge_tables(TT)
# len(data)

# dc = T.color[data.t_index]

In [7]:
lm_dir = '/pscratch/sd/n/nelfalou/lm-gaia-stars/'

brick_lateralmaps.make_lateralmaps(data, lm_dir + 'lateralmaps.fits', dc)